# Horovod + Tensorflow : Data Parallelism multi-GPU et multi-nœuds 
## Mise en pratique

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, novembre 2020*

Ce document présente la méthode à adopter sur Jean Zay pour distribuer votre entraînement Horovod avec TensorFlow 2 avec ou sans Keras, selon la méthode du ***Data Parallelism***. Il prend comme référence la [documentation Horovod](https://horovod.readthedocs.io/en/stable/) et illustre la [documentation IDRIS](http://www.idris.fr/jean-zay/gpu/jean-zay-gpu-hvd-tf-multi.html).

Dans l'exemple proposé, nous entraînons un réseau de neurones convolutionnel sur la base de données MNIST. L'apprentissage s'exécute sur plusieurs GPU et plusieurs nœuds de calcul Jean Zay.

Il s'agit ici de :
* préparer la base de données MNIST
* rédiger le script Python pour l'apprentissage distribué (Data Parallelism)
* réaliser une exécution parallèle sur Jean Zay

Il est à noter que les données MNIST et le modèle utilisé dans cet exemple sont très simples. Cela permet de présenter un code court et de tester rapidement la configuration du *Data Parallelism*, mais pas de mesurer une accélération de l'apprentissage. En effet, les temps de transfert entre GPU et le temps d'initialisation des *kernels* GPU ne sont pas négligeables par rapport aux temps d'exécution.

------------------------

### Environnement de calcul

Ce notebook est prévu pour être exécuté à partir d'une machine frontale de Jean-Zay. Le *hostname* doit être jean-zay[1-5].

In [1]:
!hostname

jean-zay2


Un module TensorFlow doit avoir été chargé pour le bon fonctionnement de ce Notebook. Par exemple, le module `tensorflow-gpu/py3/2.3.1` :

In [2]:
!module list

Currently Loaded Modulefiles:
 1) cuda/10.1.2                4) intel-compilers/19.0.4               
 2) nccl/2.5.6-2-cuda          5) openmpi/4.0.2-cuda                   
 3) cudnn/7.6.5.32-cuda-10.1   6) tensorflow-gpu/py3/2.3.1+hvd-0.21.0  
>

------------------------------------

### Préparation de la base de données MNIST

La base de données MNIST est disponible sur Jean Zay dans le DSDIR.

**Remarque** : le DSDIR, comme le SCRATCH, est un espace disque GPFS dont la bande passante est d'environ 300 Go/s en écriture et en lecture. Ils sont à privilégier pour les codes ayant une utilisation intense des opérations d'entrées/sorties. Votre space personnel SCRATCH est dédié à vos bases privées et l'espace commun DSDIR comprend la plupart des bases publiques.

Vous pouvez tester l'accès aux données avec la commande ci-dessous :

In [3]:
import os
import tensorflow as tf
import numpy as np

path = os.environ['DSDIR']+'/MNIST/mnist.npz'
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data(path)

print('Dataset MNIST\n\tNumber of datapoints: {}\n\tFile location: {}\n\tSplit: Train'.format(len(x_train), path))


Dataset MNIST
	Number of datapoints: 60000
	File location: /gpfsdswork/dataset/MNIST/mnist.npz
	Split: Train


## Horovod + TensorFlow 2

### Rédaction du script Python pour l'apprentissage distribué (Data Parallelism)

Dans cette section, nous rédigeons le script Python d'entraînement dans le fichier 'mnist-distributed.py'.

* Chargement des librairies, création de l'itérateur de données, création du modèle d'apprentissage (réseau de neurones convulationnel simple à 1 couche convolutionnel et 2 couches denses):

In [4]:
%%writefile mnist-distributed.py 

import os
import subprocess
import json
import datetime
import argparse

import tensorflow as tf
import horovod.tensorflow as hvd
import numpy as np

def mnist_dataset(batch_size):
    path = os.environ['DSDIR']+'/MNIST/mnist.npz'
    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data(path)
    # The `x` arrays are in uint8 and have values in the range [0, 255].
    # You need to convert them to float32 with values in the range [0, 1]
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).repeat().shuffle(60000).batch(batch_size)
    return train_dataset

def build_cnn_model():
    model = tf.keras.Sequential([
      tf.keras.Input(shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

    return model

Overwriting mnist-distributed.py


* Définition de la fonction d'apprentissage distribué (les *timers* et les affichages sont gérés par le *process* 0, qui est le *process* maître)

In [5]:
%%writefile -a mnist-distributed.py

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-b', '--batch-size', default=128, type =int,
                        help='batch size. it will be divided in mini-batch for each worker')
    parser.add_argument('-e','--epochs', default=2, type=int, metavar='N',
                        help='number of total epochs to run')
    args = parser.parse_args()
    
    hvd.init()
    
    # display info
    if hvd.rank() == 0:
        print(">>> Training on ", hvd.size() // hvd.local_size(), " nodes and ", hvd.size(), " processes")
    print("- Process {} corresponds to GPU {} of node {}".format(hvd.rank(), hvd.local_rank(), hvd.rank() // hvd.local_size()))
    
    # Pin GPU to be used to process local rank (one GPU per process)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    if gpus:
        tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
    
    mnist_model = build_cnn_model()
    
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Horovod: adjust learning rate based on number of GPUs.
    opt = tf.optimizers.Adam(0.001 * hvd.size())
    
    # ### Get data
    dataset = mnist_dataset(args.batch_size)
    
    @tf.function
    def training_step(images, labels, first_batch):
        with tf.GradientTape() as tape:
            probs = mnist_model(images, training=True)
            loss_value = loss(labels, probs)

        # Horovod: add Horovod Distributed GradientTape.
        tape = hvd.DistributedGradientTape(tape)

        grads = tape.gradient(loss_value, mnist_model.trainable_variables)
        opt.apply_gradients(zip(grads, mnist_model.trainable_variables))

        # Horovod: broadcast initial variable states from rank 0 to all other processes.
        # This is necessary to ensure consistent initialization of all workers when
        # training is started with random weights or restored from a checkpoint.
        #
        # Note: broadcast should be done after the first gradient step to ensure optimizer
        # initialization.
        if first_batch:
            hvd.broadcast_variables(mnist_model.variables, root_rank=0)
            hvd.broadcast_variables(opt.variables(), root_rank=0)

        return loss_value


    # Horovod: adjust number of steps based on number of GPUs.
    start = datetime.datetime.now()
    for batch, (images, labels) in enumerate(dataset.take(args.epochs * 500 // hvd.size())):
        loss_value = training_step(images, labels, batch == 0)

        if batch % 100 == 0 and hvd.rank() == 0:
            print('Step #%d\tLoss: %.6f' % (batch, loss_value))
            
    duration = datetime.datetime.now() - start

    if hvd.rank() == 0:
        print(' -- Trained in ' + str(duration) + ' -- ')


Appending to mnist-distributed.py


* Définition de la fonction principale :

In [6]:
%%writefile -a mnist-distributed.py

if __name__ == '__main__':

    main()

Appending to mnist-distributed.py


### Exemple d'exécution mono-nœud mono-GPU

* Écriture du script batch de soumission

**Rappel**:  si votre unique projet dispose d'heures CPU et GPU ou si votre login est rattaché à plusieurs projets, vous devez impérativement préciser l'attribution sur laquelle doit être décomptée les heures consommées par vos calculs, en ajoutant l'option `--account=my_project@gpu` comme indiqué dans la [documentation IDRIS](http://www.idris.fr/jean-zay/cpu/jean-zay-cpu-doc_account.html).

In [7]:
%%writefile batch_monogpu.slurm
#!/bin/sh
#SBATCH --job-name=mnist_tensorflow_monogpu
#SBATCH --output=mnist_tensorflow_monogpu.out
#SBATCH --error=mnist_tensorflow_monogpu.err
#SBATCH --ntasks=1
#SBATCH --gres=gpu:1
#SBATCH --cpus-per-task=10
#SBATCH --hint=nomultithread
##SBATCH --qos=qos_gpu-dev
#SBATCH --time=00:10:00

# go into the submission directory 
cd ${SLURM_SUBMIT_DIR}

# cleans out modules loaded in interactive and inherited by default
module purge

# loading modules
module load tensorflow-gpu/py3/2.3.1

# echo of launched commands
set -x

# code execution
srun python -u mnist-distributed.py --epochs 8 --batch-size 128

Overwriting batch_monogpu.slurm


* Soumission du script batch et affichage de la sortie

In [8]:
%%bash
# submit job
sbatch batch_monogpu.slurm

Submitted batch job 1381377


In [9]:
# watch Slurm queue line until the job is done
# execution should take about 1 minute
import time
sq = !squeue -u $USER
print(sq[0])
while len(sq) >= 2:
    print(sq[1],end='\r')
    time.sleep(5)
    sq = !squeue -u $USER
print('\n Done!')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1381377   gpu_p13 mnist_te  ssos040 CG       0:35      1 r14i4n4
 Done!


In [10]:
#display output 
%cat mnist_tensorflow_monogpu.out

>>> Training on  1  nodes and  1  processes
- Process 0 corresponds to GPU 0 of node 0
Step #0	Loss: 2.318774
Step #100	Loss: 0.294576
Step #200	Loss: 0.157830
Step #300	Loss: 0.042475
Step #400	Loss: 0.067899
Step #500	Loss: 0.052315
Step #600	Loss: 0.028083
Step #700	Loss: 0.029146
Step #800	Loss: 0.030713
Step #900	Loss: 0.043855
Step #1000	Loss: 0.029083
Step #1100	Loss: 0.010532
Step #1200	Loss: 0.049265
Step #1300	Loss: 0.009070
Step #1400	Loss: 0.008195
Step #1500	Loss: 0.031665
Step #1600	Loss: 0.025499
Step #1700	Loss: 0.021718
Step #1800	Loss: 0.021872
Step #1900	Loss: 0.007493
Step #2000	Loss: 0.007308
Step #2100	Loss: 0.011244
Step #2200	Loss: 0.007231
Step #2300	Loss: 0.015845
Step #2400	Loss: 0.002200
Step #2500	Loss: 0.000758
Step #2600	Loss: 0.010867
Step #2700	Loss: 0.021032
Step #2800	Loss: 0.002063
Step #2900	Loss: 0.005934
Step #3000	Loss: 0.011248
Step #3100	Loss: 0.007928
Step #3200	Loss: 0.000729
Step #3300	Loss: 0.002969
Step #3400	Loss: 0.012642
Step #3500	Loss

### Exemple d'exécution mono-nœud multi-GPU

* Écriture du script batch de soumission

**Rappel**:  si votre unique projet dispose d'heures CPU et GPU ou si votre login est rattaché à plusieurs projets, vous devez impérativement préciser l'attribution sur laquelle doit être décomptée les heures consommées par vos calculs, en ajoutant l'option `--account=my_project@gpu` comme indiqué dans la [documentation IDRIS](http://www.idris.fr/jean-zay/cpu/jean-zay-cpu-doc_account.html).

In [11]:
%%writefile batch_mononode.slurm
#!/bin/sh
#SBATCH --job-name=mnist_tensorflow_mononode
#SBATCH --output=mnist_tensorflow_mononode.out
#SBATCH --error=mnist_tensorflow_mononode.err
#SBATCH --ntasks=4
#SBATCH --gres=gpu:4
#SBATCH --cpus-per-task=10
#SBATCH --hint=nomultithread
##SBATCH --qos=qos_gpu-dev
#SBATCH --time=00:10:00

# go into the submission directory 
cd ${SLURM_SUBMIT_DIR}

# cleans out modules loaded in interactive and inherited by default
module purge

# loading modules
module load tensorflow-gpu/py3/2.3.1

# echo of launched commands
set -x

# code execution
srun python -u mnist-distributed.py --epochs 8 --batch-size 128

Overwriting batch_mononode.slurm


* Soumission du script batch et affichage de la sortie

In [12]:
%%bash
# submit job
sbatch batch_mononode.slurm

Submitted batch job 1381398


In [13]:
# watch Slurm queue line until the job is done
# execution should take less than 1 minute
import time
sq = !squeue -u $USER -n mnist_tensorflow_mononode
print(sq[0])
while len(sq) >= 2:
    print(sq[1],end='\r')
    time.sleep(5)
    sq = !squeue -u $USER -n mnist_tensorflow_mononode
print('\n Done!')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1381398   gpu_p13 mnist_te  ssos040 CG       0:32      1 r9i5n1
 Done!


In [14]:
#display output 
%cat mnist_tensorflow_mononode.out

>>> Training on  1  nodes and  4  processes
- Process 3 corresponds to GPU 3 of node 0
- Process 0 corresponds to GPU 0 of node 0
- Process 1 corresponds to GPU 1 of node 0
- Process 2 corresponds to GPU 2 of node 0
Step #0	Loss: 2.306480
Step #100	Loss: 0.043011
Step #200	Loss: 0.053806
Step #300	Loss: 0.023688
Step #400	Loss: 0.026603
Step #500	Loss: 0.017523
Step #600	Loss: 0.009134
Step #700	Loss: 0.012096
Step #800	Loss: 0.000577
Step #900	Loss: 0.007398
 -- Trained in 0:00:11.279000 -- 


### Exemple d'exécution multi-nœuds multi-GPU

* Écriture du script batch de soumission

**Rappel**:  si votre unique projet dispose d'heures CPU et GPU ou si votre login est rattaché à plusieurs projets, vous devez impérativement préciser l'attribution sur laquelle doit être décomptée les heures consommées par vos calculs, en ajoutant l'option `--account=my_project@gpu` comme indiqué dans la [documentation IDRIS](http://www.idris.fr/jean-zay/cpu/jean-zay-cpu-doc_account.html).

In [15]:
%%writefile batch_multinode.slurm
#!/bin/sh
#SBATCH --job-name=mnist_tensorflow_multinode
#SBATCH --output=mnist_tensorflow_multinode.out
#SBATCH --error=mnist_tensorflow_multinode.err
#SBATCH --nodes=3
#SBATCH --ntasks-per-node=4
#SBATCH --gres=gpu:4
#SBATCH --cpus-per-task=10
#SBATCH --hint=nomultithread
##SBATCH --qos=qos_gpu-dev
#SBATCH --time=00:10:00


# go into the submission directory 
cd ${SLURM_SUBMIT_DIR}

# cleans out modules loaded in interactive and inherited by default
module purge

# loading modules
module load tensorflow-gpu/py3/2.2.0

# echo of launched commands
set -x

# code execution
srun python -u mnist-distributed.py --epochs 8 --batch-size 128

Overwriting batch_multinode.slurm


* Soumission du script batch et affichage de la sortie

In [16]:
%%bash
# submit job
sbatch batch_multinode.slurm

Submitted batch job 1381428


sbatch: IDRIS: setting exclusive mode for the job.


In [17]:
# watch Slurm queue line until the job is done
# execution should take about 1 minute
import time
sq = !squeue -u $USER -n mnist_tensorflow_multinode
print(sq[0])
while len(sq) == 2:
    print(sq[1],end='\r')
    time.sleep(5)
    sq = !squeue -u $USER -n mnist_tensorflow_multinode
print('\n Done!')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1381428   gpu_p13 mnist_te  ssos040  R       0:40      3 r10i3n[2-4]
 Done!


In [18]:
# display output
%cat mnist_tensorflow_multinode.out

- Process 8 corresponds to GPU 0 of node 2
- Process 9 corresponds to GPU 1 of node 2
- Process 11 corresponds to GPU 3 of node 2
- Process 6 corresponds to GPU 2 of node 1
>>> Training on  3  nodes and  12  processes
- Process 10 corresponds to GPU 2 of node 2
- Process 4 corresponds to GPU 0 of node 1
- Process 2 corresponds to GPU 2 of node 0
- Process 5 corresponds to GPU 1 of node 1
- Process 3 corresponds to GPU 3 of node 0
- Process 7 corresponds to GPU 3 of node 1
- Process 0 corresponds to GPU 0 of node 0
- Process 1 corresponds to GPU 1 of node 0
Step #0	Loss: 2.303728
Step #100	Loss: 0.039106
Step #200	Loss: 0.021328
Step #300	Loss: 0.002840
 -- Trained in 0:00:08.967639 -- 


## Horovod + TensorFlow 2 avec Keras

Dans cette section, nous rédigeons le script Python d'entraînement dans le fichier 'mnist-distributed.py'.

* Chargement des librairies, création de l'itérateur de données, création du modèle d'apprentissage (réseau de neurones convulationnel simple à 1 couche convolutionnel et 2 couches denses):

In [19]:
%%writefile mnist-distributed.py 

import os
import subprocess
import json
import datetime
import argparse

import tensorflow as tf
import horovod.tensorflow.keras as hvd
import numpy as np

def mnist_dataset(batch_size):
    path = os.environ['SCRATCH']+'/MNIST/mnist.npz'
    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data(path)
    # The `x` arrays are in uint8 and have values in the range [0, 255].
    # You need to convert them to float32 with values in the range [0, 1]
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).repeat().shuffle(60000).batch(batch_size)
    return train_dataset

def build_cnn_model():
    model = tf.keras.Sequential([
      tf.keras.Input(shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

    return model

Overwriting mnist-distributed.py


* Définition de la fonction d'apprentissage distribué (les *timers* et les affichages sont gérés par le *process* 0, qui est le *process* maître)

In [20]:
%%writefile -a mnist-distributed.py

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-b', '--batch-size', default=128, type =int,
                        help='batch size. it will be divided in mini-batch for each worker')
    parser.add_argument('-e','--epochs', default=2, type=int, metavar='N',
                        help='number of total epochs to run')
    args = parser.parse_args()
    
    hvd.init()
    
    # display info
    if hvd.rank() == 0:
        print(">>> Training on ", hvd.size() // hvd.local_size(), " nodes and ", hvd.size(), " processes")
    print("- Process {} corresponds to GPU {} of node {}".format(hvd.rank(), hvd.local_rank(), hvd.rank() // hvd.local_size()))
    
    # Pin GPU to be used to process local rank (one GPU per process)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    if gpus:
        tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

    # Horovod: adjust learning rate based on number of GPUs.
    scaled_lr = 0.001 * hvd.size()
    opt = tf.optimizers.Adam(scaled_lr)
    
    # Horovod: add Horovod DistributedOptimizer.
    opt = hvd.DistributedOptimizer(opt)
    
    model = build_cnn_model()
    
    callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
    ]

    # Horovod: Specify `experimental_run_tf_function=False` to ensure TensorFlow
    # uses hvd.DistributedOptimizer() to compute gradients.
    model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=opt,
      metrics=['accuracy'],
      experimental_run_tf_function=False)

    # ### Get data

    multi_worker_dataset = mnist_dataset(args.batch_size)

    # ### Train the model using "fit" method
    start = datetime.datetime.now()
    # Train the model.
    # Horovod: adjust number of steps based on number of GPUs.
    model.fit(multi_worker_dataset,
              steps_per_epoch=500 // hvd.size(),
              epochs=args.epochs,
              callbacks=callbacks,
              verbose=1 if hvd.rank() == 0 else 0)
    duration = datetime.datetime.now() - start

    if hvd.rank() == 0:
        print(' -- Trained in ' + str(duration) + ' -- ')
    

Appending to mnist-distributed.py


* Définition de la fonction principale :

In [21]:
%%writefile -a mnist-distributed.py

if __name__ == '__main__':

    main()

Appending to mnist-distributed.py


### Exemple d'exécution mono-nœud mono-GPU

* Écriture du script batch de soumission

**Rappel**:  si votre unique projet dispose d'heures CPU et GPU ou si votre login est rattaché à plusieurs projets, vous devez impérativement préciser l'attribution sur laquelle doit être décomptée les heures consommées par vos calculs, en ajoutant l'option `--account=my_project@gpu` comme indiqué dans la [documentation IDRIS](http://www.idris.fr/jean-zay/cpu/jean-zay-cpu-doc_account.html).

In [22]:
%%writefile batch_monogpu.slurm
#!/bin/sh
#SBATCH --job-name=mnist_tensorflow_monogpu
#SBATCH --output=mnist_tensorflow_monogpu.out
#SBATCH --error=mnist_tensorflow_monogpu.err
#SBATCH --ntasks=1
#SBATCH --gres=gpu:1
#SBATCH --cpus-per-task=10
#SBATCH --hint=nomultithread
##SBATCH --qos=qos_gpu-dev
#SBATCH --time=00:10:00

# go into the submission directory 
cd ${SLURM_SUBMIT_DIR}

# cleans out modules loaded in interactive and inherited by default
module purge

# loading modules
module load tensorflow-gpu/py3/2.3.1

# echo of launched commands
set -x

# code execution
srun python -u mnist-distributed.py --epochs 8 --batch-size 128

Overwriting batch_monogpu.slurm


* Soumission du script batch et affichage de la sortie

%%bash
# submit job
sbatch batch_monogpu.slurm

**Rappel**:  si votre unique projet dispose d'heures CPU et GPU ou si votre login est rattaché à plusieurs projets, vous devez impérativement préciser l'attribution sur laquelle doit être décomptée les heures consommées par vos calculs, en ajoutant l'option `--account=my_project@gpu` comme indiqué dans la [documentation IDRIS](http://www.idris.fr/jean-zay/cpu/jean-zay-cpu-doc_account.html).

In [24]:
# watch Slurm queue line until the job is done
# execution should take about 1 minute
import time
sq = !squeue -u $USER
print(sq[0])
while len(sq) >= 2:
    print(sq[1],end='\r')
    time.sleep(5)
    sq = !squeue -u $USER
print('\n Done!')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1381475   gpu_p13 mnist_te  ssos040  R       0:25      1 r13i7n1
 Done!


In [25]:
# display output
%cat mnist_tensorflow_monogpu.out

>>> Training on  1  nodes and  1  processes
- Process 0 corresponds to GPU 0 of node 0
Epoch 1/8
500/500 [==============================] - 1s 3ms/step - loss: 0.1652 - accuracy: 0.9516
Epoch 2/8
500/500 [==============================] - 1s 3ms/step - loss: 0.0499 - accuracy: 0.9859
Epoch 3/8
500/500 [==============================] - 1s 3ms/step - loss: 0.0288 - accuracy: 0.9913
Epoch 4/8
500/500 [==============================] - 1s 3ms/step - loss: 0.0177 - accuracy: 0.9948
Epoch 5/8
500/500 [==============================] - 1s 3ms/step - loss: 0.0116 - accuracy: 0.9965
Epoch 6/8
500/500 [==============================] - 1s 3ms/step - loss: 0.0085 - accuracy: 0.9974
Epoch 7/8
500/500 [==============================] - 1s 3ms/step - loss: 0.0068 - accuracy: 0.9980
Epoch 8/8
500/500 [==============================] - 1s 3ms/step - loss: 0.0050 - accuracy: 0.9985
 -- Trained in 0:00:13.589227 -- 


### Exemple d'exécution mono-nœud multi-GPU

* Écriture du script batch de soumission

**Rappel**:  si votre unique projet dispose d'heures CPU et GPU ou si votre login est rattaché à plusieurs projets, vous devez impérativement préciser l'attribution sur laquelle doit être décomptée les heures consommées par vos calculs, en ajoutant l'option `--account=my_project@gpu` comme indiqué dans la [documentation IDRIS](http://www.idris.fr/jean-zay/cpu/jean-zay-cpu-doc_account.html).

In [26]:
%%writefile batch_mononode.slurm
#!/bin/sh
#SBATCH --job-name=mnist_tensorflow_mononode
#SBATCH --output=mnist_tensorflow_mononode.out
#SBATCH --error=mnist_tensorflow_mononode.err
#SBATCH --ntasks=4
#SBATCH --gres=gpu:4
#SBATCH --cpus-per-task=10
#SBATCH --hint=nomultithread
##SBATCH --qos=qos_gpu-dev
#SBATCH --time=00:10:00

# go into the submission directory 
cd ${SLURM_SUBMIT_DIR}

# cleans out modules loaded in interactive and inherited by default
module purge

# loading modules
module load tensorflow-gpu/py3/2.3.1

# echo of launched commands
set -x

# code execution
srun python -u mnist-distributed.py --epochs 8 --batch-size 128

Overwriting batch_mononode.slurm


* Soumission du script batch et affichage de la sortie

In [27]:
%%bash
# submit job
sbatch batch_mononode.slurm

Submitted batch job 1381497


In [28]:
# watch Slurm queue line until the job is done
# execution should take less than 1 minute
import time
sq = !squeue -u $USER -n mnist_tensorflow_mononode
print(sq[0])
while len(sq) >= 2:
    print(sq[1],end='\r')
    time.sleep(5)
    sq = !squeue -u $USER -n mnist_tensorflow_mononode
print('\n Done!')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1381497   gpu_p13 mnist_te  ssos040  R       0:13      1 r8i0n3
 Done!


In [29]:
#display output 
%cat mnist_tensorflow_mononode.out

- Process 1 corresponds to GPU 1 of node 0
- Process 3 corresponds to GPU 3 of node 0
>>> Training on  1  nodes and  4  processes
- Process 0 corresponds to GPU 0 of node 0
- Process 2 corresponds to GPU 2 of node 0
Epoch 1/8
125/125 [==============================] - 1s 5ms/step - loss: 0.1969 - accuracy: 0.9391
Epoch 2/8
125/125 [==============================] - 1s 5ms/step - loss: 0.0438 - accuracy: 0.9882
Epoch 3/8
125/125 [==============================] - 1s 5ms/step - loss: 0.0264 - accuracy: 0.9924
Epoch 4/8
125/125 [==============================] - 1s 5ms/step - loss: 0.0171 - accuracy: 0.9951
Epoch 5/8
125/125 [==============================] - 1s 5ms/step - loss: 0.0111 - accuracy: 0.9967
Epoch 6/8
125/125 [==============================] - 1s 5ms/step - loss: 0.0044 - accuracy: 0.9987
Epoch 7/8
125/125 [==============================] - 1s 5ms/step - loss: 0.0043 - accuracy: 0.9986
Epoch 8/8
125/125 [==============================] - 1s 5ms/step - loss: 0.0077 - accuracy:

### Exemple d'exécution multi-nœuds multi-GPU

* Écriture du script batch de soumission

**Rappel**:  si votre unique projet dispose d'heures CPU et GPU ou si votre login est rattaché à plusieurs projets, vous devez impérativement préciser l'attribution sur laquelle doit être décomptée les heures consommées par vos calculs, en ajoutant l'option `--account=my_project@gpu` comme indiqué dans la [documentation IDRIS](http://www.idris.fr/jean-zay/cpu/jean-zay-cpu-doc_account.html).

In [30]:
%%writefile batch_multinode.slurm
#!/bin/sh
#SBATCH --job-name=mnist_tensorflow_multinode
#SBATCH --output=mnist_tensorflow_multinode.out
#SBATCH --error=mnist_tensorflow_multinode.err
#SBATCH --nodes=3
#SBATCH --ntasks-per-node=4
#SBATCH --gres=gpu:4
#SBATCH --cpus-per-task=10
#SBATCH --hint=nomultithread
##SBATCH --qos=qos_gpu-dev
#SBATCH --time=00:10:00

# go into the submission directory 
cd ${SLURM_SUBMIT_DIR}

# cleans out modules loaded in interactive and inherited by default
module purge

# loading modules
module load tensorflow-gpu/py3/2.2.0

# echo of launched commands
set -x

# code execution
srun python -u mnist-distributed.py --epochs 8 --batch-size 128

Overwriting batch_multinode.slurm


* Soumission du script batch et affichage de la sortie

In [31]:
%%bash
# submit job
sbatch batch_multinode.slurm

Submitted batch job 1381500


sbatch: IDRIS: setting exclusive mode for the job.


In [32]:
# watch Slurm queue line until the job is done
# execution should take about 1 minute
import time
sq = !squeue -u $USER -n mnist_tensorflow_multinode
print(sq[0])
while len(sq) == 2:
    print(sq[1],end='\r')
    time.sleep(5)
    sq = !squeue -u $USER -n mnist_tensorflow_multinode
print('\n Done!')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1381500   gpu_p13 mnist_te  ssos040  R       0:15      3 r8i0n3,r8i3n[0-1]
 Done!


In [33]:
# display output
%cat mnist_tensorflow_multinode.out

- Process 2 corresponds to GPU 2 of node 0
- Process 3 corresponds to GPU 3 of node 0
- Process 1 corresponds to GPU 1 of node 0
- Process 4 corresponds to GPU 0 of node 1
- Process 11 corresponds to GPU 3 of node 2
>>> Training on  3  nodes and  12  processes
- Process 5 corresponds to GPU 1 of node 1
- Process 8 corresponds to GPU 0 of node 2
- Process 0 corresponds to GPU 0 of node 0
- Process 6 corresponds to GPU 2 of node 1
- Process 9 corresponds to GPU 1 of node 2
- Process 7 corresponds to GPU 3 of node 1
- Process 10 corresponds to GPU 2 of node 2
Epoch 1/8
41/41 [==============================] - 0s 10ms/step - loss: 0.6382 - accuracy: 0.8178
Epoch 2/8
41/41 [==============================] - 0s 10ms/step - loss: 0.0741 - accuracy: 0.9771
Epoch 3/8
41/41 [==============================] - 0s 10ms/step - loss: 0.0436 - accuracy: 0.9874
Epoch 4/8
41/41 [==============================] - 0s 10ms/step - loss: 0.0301 - accuracy: 0.9912
Epoch 5/8
41/41 [============================